In [2]:
import json
import os
import csv
import re
import random
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from tqdm import tqdm
from datasets import Dataset, DatasetDict

In [3]:
def avg(_list):
    return sum(_list) / len(_list)

def json_reader(file_name):
    f = open(file_name)
    data = json.load(f)
    f.close()
    return data

def csv_reader(file_name):
    f = open(file_name, 'r')
    reader = csv.reader(f)
    # f.close()
    return reader

def convert_to_lowercase(input_string):
    return input_string.lower()

def has_alphabets(input_string):
    return any(char.isalpha() for char in input_string)

def has_numbers(input_string):
    has_numbers = any(char.isdigit() for char in input_string)
    return has_numbers

def extract_words_inside_brackets(input_string):
    pattern = r"\[\'([^\']+)\'\]|\[\"([^\"]+)\"\]"

    matches = re.findall(pattern, input_string)

    extracted_words = ' '.join([word for match in matches for word in match if word])

    return extracted_words
     

In [4]:
data_local_path_header = r'C:\Users\35950\Downloads\Dataset-SLM\\'

MODE = 'train'  # options: train, test, validation
DATA = 'PMData'  # options: PMData, other datasets

In [7]:
if MODE == "train":
    if DATA == "PMData":
        print("[INFO] Generating datasets for PMData ...")
        """ 
            1) Readiness Prediction 
            2) Stress Estimation
            3) Calorie Consumption Prediction (food image) 
            4) Sleep Quality Prediction
        """
        participant_info = {
            'p01': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p02': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p03': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p04': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p05': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p06': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p07': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p08': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p09': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p10': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p11': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p12': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p13': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p14': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p15': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p16': [-1, -1, 'N/A', 'N/A', 'N/A'],
        }
        
        DATA_PATH = data_local_path_header + "Dataset-SLM\pmdata"
        final_data_pmdata = []
        
        fatigue_records = []
        readiness_records = []
        stress_records = []
        sleep_quality_records = []
        for SUBTASK in ['sleep_quality', 'stress', 'readiness', 'fatigue']:
            index = 0
            subtask_data = []
            true_labels = []
            print("[INFO] Subtask:", SUBTASK)
            for dir1 in tqdm(os.listdir(DATA_PATH)):
                print(dir1)
                if "." in dir1:
                    continue
                tmp = participant_info[dir1]
                age = tmp[0]
                height = str(tmp[1]) + " cm"
                gender = tmp[2]
                fpath1 = os.path.join(DATA_PATH, dir1)

                if '.' not in dir1:      
                    # print("[INFO] Participant:", dir1)      
                    for dir2 in os.listdir(fpath1):
                        fpath2 = os.path.join(fpath1, dir2)

                        if dir2 == 'fitbit':
                            # [1] fitbit
                            # 1-1) calories.json
                            # 1-2) distance.json
                            
                            # 1-3) exercise.json
                            exercise_data = json_reader(fpath2 + '/exercise.json')
                            
                            # 1-4) heart_rate.json
                            try:
                                heart_rate_data = json_reader(fpath2 + '/resting_heart_rate.json')
                            except:
                                continue
                            
                            # 1-5) sleep_score.csv
                            # 1-6) sleep.json
                            sleep_data = json_reader(fpath2 + '/sleep.json')

                            # 1-7) steps.json
                            # 1-8) time_in_heart_rate_zones.json
                        
                        elif dir2 == 'pmsys':

                            wellness_data = csv_reader(fpath2 + "/wellness.csv")
                            wellness_dict = {k:[] for k in ['effective_time_frame', 'fatigue', 'mood', 'readiness', 'sleep_duration_h', 'sleep_quality', 'soreness', 'soreness_area', 'stress']}
                            for i,data in enumerate(wellness_data):
                                # ['effective_time_frame', 'fatigue', 'mood', 'readiness', 'sleep_duration_h', 'sleep_quality', 'soreness', 'soreness_area', 'stress']
                                if i == 0:
                                    continue

                                date = data[0][:10] + "_" + data[0][11:][:-1].split(".")[0]
                                fatigue = data[1]
                                mood = data[2]
                                readiness = data[3]
                                sleep_dur = data[4]
                                sleep_qual = data[5]
                                # soreness = data[6]
                                # soreness_area = data[7]
                                stress = data[-1]

                                wellness_dict['effective_time_frame'].append(date)
                                wellness_dict['fatigue'].append(fatigue)
                                wellness_dict['mood'].append(mood)
                                wellness_dict['readiness'].append(readiness)
                                wellness_dict['sleep_duration_h'].append(sleep_dur)
                                wellness_dict['sleep_quality'].append(sleep_qual)
                                wellness_dict['stress'].append(stress)

                    
                    date = wellness_dict['effective_time_frame']
                    fatigue = wellness_dict['fatigue']
                    mood = wellness_dict['mood']
                    readiness = wellness_dict['readiness']
                    sleep_dur = wellness_dict['sleep_duration_h']
                    sleep_qual = wellness_dict['sleep_quality']
                    stress = wellness_dict['stress']

                    
                    for d,f,m,r,sd,sq,s in zip(date, fatigue, mood, readiness, sleep_dur, sleep_qual, stress):
                        # print("[INFO] Readiness:", r)
                        new_d = datetime.strptime(d, '%Y-%m-%d_%H:%M:%S')
                        # print("[INFO] target date:", new_d)

                        # [1] mood 
                        mood = m
                        stress = s
                        sleep_quality = sq
                        
                        # [2] exercise
                        exercise_hist = []
                        while True:
                            # TODO: datetime compare / 7-days or 14-days? -> refer to fitbit premium / heart rate
                            for e_data in exercise_data:
                                # print(e_data)
                                # print()
                                e_date = e_data['startTime'][:10] + "_" + e_data['startTime'][11:]
                                new_ed = datetime.strptime(e_date, '%Y-%m-%d_%H:%M:%S')
                                # print("[INFO] e_data's date:", new_ed)

                                # compare the date and make 2-weeks window
                                if (new_d > new_ed) and (new_d - new_ed) < timedelta(days=14):

                                    try:
                                        activity = e_data['activityName']
                                        burn_calories = float(e_data['calories'])
                                        steps = float(e_data['steps'])
                                        duration = float(e_data['duration']) / 1000 / 60 # in minutes
                                        exercise_hist.append([new_ed, activity, duration, burn_calories, steps])
                                    except:
                                        continue
                                    
                            break                 
                        
                        # [3] sleep
                        sleep_hist = []
                        while True:
                            for s_data in sleep_data:
                                # print(s_data)
                                # print()
                                s_date = s_data['startTime'][:10] + "_" + s_data['startTime'][11:]
                                new_sd = datetime.strptime(s_date, '%Y-%m-%d_%H:%M:%S')

                                # compare the date and make 2-weeks window
                                if (new_d > new_sd) and (new_d - new_sd) < timedelta(days=14):
                                    # print("[INFO2]", new_sd, "belongs")

                                    sleep_duration = float(s_data['duration']) / 1000 / 60 # in minutes
                                    sleep_hist.append([new_sd, sleep_duration])

                            break

                        # [4] heart rate
                        hr_hist = []
                        while True:
                            for hr_data in heart_rate_data:
                                # print(hr_data)
                                hr_date = hr_data['dateTime'][:10] + "_" + hr_data['dateTime'][11:]
                                new_hrd = datetime.strptime(hr_date, '%Y-%m-%d_%H:%M:%S')
                                # print("[INFO] e_data's date:", new_ed)
    
                                # compare the date and make 2-weeks window
                                if (new_d > new_hrd) and (new_d - new_hrd) < timedelta(days=14):
                                    # print("[INFO3]", new_hrd, "belongs")
                                    rhr = float(hr_data['value']['value'])
                                    hr_hist.append([new_hrd, rhr])
                            break

                        # encode historical data
                        try:
                            steps_14d = sum([x[-1] for x in exercise_hist]) / len(exercise_hist)
                        except:
                            steps_14d = "N/A"
                            continue
                        try:
                            calories_14d = sum([x[-2] for x in exercise_hist]) / len(exercise_hist)
                        except:
                            calories_14d = "N/A"
                            continue
                        try:
                            rhr_14d = sum([x[-1] for x in hr_hist]) / len(hr_hist)
                        except:
                            rhr_14d = "N/A"
                            continue
                        try:
                            sleep_dur_14d = sum([x[-1] for x in sleep_hist]) / len(sleep_hist)
                        except:
                            sleep_dur_14d = "N/A"
                            continue

                        if SUBTASK == "readiness":
                            range1 = 0
                            range2 = 10
                        elif SUBTASK == "stress":
                            range1 = 1
                            range2 = 5
                        elif SUBTASK == "sleep_quality":
                            range1 = 1
                            range2 = 5 
                        elif SUBTASK == "fatigue":
                            range1 = 1
                            range2 = 5

                        # few-shot chain of thoughts context enhancement
                        # I = "You are a personalized healthcare agent trained to predict {} which ranges from {} to {} based on physiological data and user information.".format(SUBTASK, range1, range2)        
                        Q = "The recent 14-days sensor readings show: [Steps]: {} steps, [Burned Calorories]: {} calories, [Resting Heart Rate]: {} beats/min, [SleepMinutes]: {} minutes, [Mood]: {} out of 5; What would be the predicted {}?".format([x[-1] for x in exercise_hist], [x[-2] for x in exercise_hist], [x[-1] for x in hr_hist], [x[-1] for x in sleep_hist], m, SUBTASK)
                        # I = "You are a personalized healthcare agent trained to predict {} which ranges from 1 to 5 based on physiological data and user information."
                
                        # NEW
                        if SUBTASK == "readiness":
                            # zero-shot context giving - instruction style
                            # I = "Given the user's context and sensor data, predict the readiness score (ranges from 0 to 10)"
                            I = "You are a personalized healthcare agent trained to predict readiness score which ranges from 0 to 10 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, r)
                            # readiness_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Readiness':r})

                        elif SUBTASK == "stress":
                            # I = "Given the user's context and sensor data, predict the stress level (ranges from 0 to 5)"
                            I = "You are a personalized healthcare agent trained to predict stress level which ranges from 0 to 5 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, s)
                            # stress_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Stress':s})

                        elif SUBTASK == "sleep_quality":
                            # I = "Given the user's context and sensor data, predict the sleep quality (ranges from 1 to 5)"
                            I = "You are a personalized healthcare agent trained to predict sleep quality which ranges from 1 to 5 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, sq)
                            # sleep_quality_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Sleep_quality':sq})
                            
                        elif SUBTASK == "fatigue":
                            # I = "Given the user's context and sensor data, predict the fatigue score (ranges from 0 to 5)"
                            I = "You are a personalized healthcare agent trained to predict fatigue score which ranges from 0 to 5 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, f)
                            # fatigue_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Fatigue':f})

                        subtask_data.append({'index':index,'task':SUBTASK, 'instruction':I, 'input':Q, 'output':A})
                        index += 1
            final_data_pmdata.append(subtask_data)
            

            

<>:29: SyntaxWarning: invalid escape sequence '\p'
<>:29: SyntaxWarning: invalid escape sequence '\p'
C:\Users\35950\AppData\Local\Temp\ipykernel_16124\1465149161.py:29: SyntaxWarning: invalid escape sequence '\p'
  DATA_PATH = data_local_path_header + "Dataset-SLM\pmdata"


[INFO] Generating datasets for PMData ...
[INFO] Subtask: sleep_quality


  0%|                                                                                           | 0/17 [00:00<?, ?it/s]

p01


  6%|████▉                                                                              | 1/17 [00:00<00:07,  2.19it/s]

...

p14


 82%|███████████████████████████████████████████████████████████████████▌              | 14/17 [00:04<00:00,  3.59it/s]

p15


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  3.58it/s]

p16
participant-overview.xlsx


In [12]:
def convert_to_rlcr_format(data_list, task_name):
    """
    Convert your custom data format into RLCR format.
    
    Input format (per item):
    {
        'index': 0,
        'task': 'sleep_quality',
        'instruction': '...',
        'input': '...',
        'output': 'The predicted sleep_quality level is 3.'
    }
    
    Output format (per item):
    {
        'problem': '...',
        'answer': '3',
        'source': 'PMData-sleep_quality',
        'index': 0
    }
    """
    converted_data = []
    
    for item in data_list:
        # 1. Merge instruction and input as the 'problem'
        problem = f"{item['instruction']}\n\n{item['input']}"
        
        # 2. Extract the numeric answer from the output text
        output_text = item['output']
        # First, try to match "level is X."
        answer_match = re.search(r'level is (\d+)', output_text)
        if answer_match:
            answer = answer_match.group(1)
        else:
            # If that fails, fall back to any digit sequence in the text
            answer_match = re.search(r'(\d+)', output_text)
            answer = answer_match.group(1) if answer_match else "3"
        
        # 3. Build a single RLCR-format record
        converted_item = {
            'problem': problem,
            'answer': answer,
            'source': "PMData" + "-" + task_name,  # used to choose evaluation method
            'index': item['index'],
        }
        
        converted_data.append(converted_item)
    
    return converted_data


# Convert all subtasks
# final_data_list is the list produced by your previous code.
# final_data_list[0] = sleep_quality data
# final_data_list[1] = stress data
# final_data_list[2] = readiness data  
# final_data_list[3] = fatigue data

task_names = ['sleep_quality', 'stress', 'readiness', 'fatigue']
converted_datasets = {}

for i, (task_data, task_name) in enumerate(zip(final_data_pmdata, task_names)):
    print(f"\n{'='*60}")
    print(f"task_name: {task_name}")
    print(f"Original data size: {len(task_data)}")
    
    # Convert to RLCR format
    converted_data = convert_to_rlcr_format(task_data, task_name)
    
    print(f"Converted data size: {len(converted_data)}")
    print(f"\nFirst example:")
    print(f"Problem (first 200 chars): {converted_data[0]['problem'][:200]}...")
    print(f"Answer: {converted_data[0]['answer']}")
    print(f"Source: {converted_data[0]['source']}")
    
    converted_datasets[task_name] = converted_data

print(f"\n{'='*60}")
print("✅ All tasks converted to RLCR format!")


# =========================
# Save each task as a CSV
# =========================

# Option 1: use HuggingFace Datasets to write CSV files
for task_name in task_names:
    data = converted_datasets[task_name]
    ds = Dataset.from_list(data)
    csv_filename = f"C:\\Users\\35950\\Downloads\\Dataset-SLM\\Processed_Data\\PMData_{task_name}.csv"
    ds.to_csv(csv_filename)
    print(f"💾 Saved {csv_filename} with {len(data)} rows.")

# If you prefer pandas instead, you can use:
# import pandas as pd
# for task_name in task_names:
#     data = converted_datasets[task_name]
#     df = pd.DataFrame(data)
#     csv_filename = f"PMData_{task_name}.csv"
#     df.to_csv(csv_filename, index=False)
#     print(f"💾 Saved {csv_filename} with {len(df)} rows.")



task_name: sleep_quality
Original data size: 1380
Converted data size: 1380

First example:
Problem (first 200 chars): You are a personalized healthcare agent trained to predict sleep quality which ranges from 1 to 5 based on physiological data and user information.

The recent 14-days sensor readings show: [Steps]: [...
Answer: 3
Source: PMData-sleep_quality

task_name: stress
Original data size: 1380
Converted data size: 1380

First example:
Problem (first 200 chars): You are a personalized healthcare agent trained to predict stress level which ranges from 0 to 5 based on physiological data and user information.

The recent 14-days sensor readings show: [Steps]: [1...
Answer: 3
Source: PMData-stress

task_name: readiness
Original data size: 1380
Converted data size: 1380

First example:
Problem (first 200 chars): You are a personalized healthcare agent trained to predict readiness score which ranges from 0 to 10 based on physiological data and user information.

The recent 14-days s

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\PMData_sleep_quality.csv with 1380 rows.


Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\PMData_stress.csv with 1380 rows.


Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\PMData_readiness.csv with 1380 rows.


Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\PMData_fatigue.csv with 1380 rows.


In [5]:
# SUBTASK = "anxiety" #"depression", "anxiety"
subtasks = ["anxiety", "depression"]
# print("[INFO] Dataset:", DATA)
# print("[INFO] Subtask:", SUBTASK)
# TODO: add INS-W_2,3,4
final_data_globem = []
for SUBTASK in subtasks:
    final_data = []
    for insw_idx in tqdm([1,2,3,4]):
        data1 = csv_reader(data_local_path_header + 'Dataset-SLM/globem/1.1/INS-W_{}/SurveyData/dep_weekly.csv'.format(insw_idx))
        dict1 = {}
        keys1 = []

        for idx,_data in enumerate(data1):
            # ['', 'pid', 'date', 'feel_anxious', 'feel_depressed', 'BDI2', 'dep', 'dep_weekly_subscale', 'anx_weekly_subscale', 'dep_weeklysubscale_endterm_merged']
            if idx == 0:
                for _idx,_d in enumerate(_data):
                    if _idx == 0:
                        continue
                    keys1.append(_d)
                    dict1[_d] = []
            else:
                for _idx, _d in enumerate(_data):
                    dict1[keys1[_idx-1]].append(_d)
        
        data2 = csv_reader(data_local_path_header + 'Dataset-SLM/globem/1.1/INS-W_{}/FeatureData/steps.csv'.format(insw_idx))
        dict2 = {}
        keys2 = []

        for idx,_data in enumerate(data2):
            if idx == 0:
                for _idx,_d in enumerate(_data):
                    if _idx == 0:
                        continue
                    keys2.append(_d)
                    dict2[_d] = []
            else:
                for _idx, _d in enumerate(_data):
                    dict2[keys2[_idx-1]].append(_d)
        
        data3 = csv_reader(data_local_path_header + 'Dataset-SLM/globem/1.1/INS-W_{}/FeatureData/sleep.csv'.format(insw_idx))
        dict3 = {}
        keys3 = []

        for idx,_data in enumerate(data3):
            if idx == 0:
                for _idx,_d in enumerate(_data):
                    if _idx == 0:
                        continue
                    keys3.append(_d)
                    dict3[_d] = []
            else:
                for _idx, _d in enumerate(_data):
                    dict3[keys3[_idx-1]].append(_d)
        
        for idx,pid in enumerate(dict1['pid']):
            skip = False
            date = dict1['date'][idx]
            try:
                dep = float(dict1['feel_depressed'][idx])
                anx = float(dict1['feel_anxious'][idx])
                # print("[{}]".format(idx))
                # print("[dict1] pid: {}, date: {}, dep: {:.2f}".format(pid, date, dep))
            except:
                continue
            
            i = 0
            while True:
                if dict2['pid'][i] == pid and dict2['date'][i] == date:
                    interests = [
                        "f_steps:fitbit_steps_summary_rapids_maxsumsteps:14dhist", # The maximum daily step count during a time segment.
                        "f_steps:fitbit_steps_summary_rapids_minsumsteps:14dhist", 
                        "f_steps:fitbit_steps_summary_rapids_avgsumsteps:14dhist", 
                        "f_steps:fitbit_steps_summary_rapids_mediansumsteps:14dhist", 
                        "f_steps:fitbit_steps_summary_rapids_stdsumsteps:14dhist" 
                    ]
                    try:
                        maxsumsteps = float(dict2[interests[0]][i])
                        minsumsteps = float(dict2[interests[1]][i])
                        avgsumsteps = float(dict2[interests[2]][i])
                        mediansumsteps = float(dict2[interests[3]][i])
                        stdsumsteps = float(dict2[interests[4]][i])

                        # print("[dict2] pid: {}, date: {}, max: {:.2f}, min: {:.2f}, avg: {:.2f}, median: {:.2f}, std: {:.2f}".format(dict2['pid'][i], dict2['date'][i], maxsumsteps, minsumsteps, avgsumsteps, mediansumsteps, stdsumsteps))
                    except:
                        skip = True
                        break 
                    
                    break
                else:
                    i += 1
            
            i = 0
            while True:
                if dict3['pid'][i] == pid and dict3['date'][i] == date:
                    interests = [
                        "f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:14dhist", # Average sleep efficiency for a certain sleep type during a time segment.
                        "f_slp:fitbit_sleep_summary_rapids_avgdurationafterwakeupmain:14dhist", # Average duration the user stayed in bed after waking up for a certain sleep type during a time segment.
                        "f_slp:fitbit_sleep_summary_rapids_avgdurationasleepmain:14dhist", # Average duration the user spent to sleep for a certain sleep type during a time segment.
                        "f_slp:fitbit_sleep_summary_rapids_avgdurationawakemain:14dhist", # Average duration the user stayed awake but still in bed for a certain sleep type during a time segment.
                        "f_slp:fitbit_sleep_summary_rapids_avgdurationtofallasleepmain:14dhist", # Average duration the user spent to fall asleep for a certain sleep type during a time segment.
                        "f_slp:fitbit_sleep_summary_rapids_avgdurationinbedmain:14dhist" 
                    ]
                    try:
                        durafwake = float(dict3[interests[1]][i])
                        dursleep = float(dict3[interests[2]][i])
                        durawake = float(dict3[interests[3]][i])
                        durfall = float(dict3[interests[4]][i])
                        durbed = float(dict3[interests[5]][i])
                        eff = float(dict3[interests[0]][i])

                        # print("[dict3] pid: {}, date: {}, eff: {:.2f}, durafwake: {:.2f}, dursleep: {:.2f}, durawake: {:.2f}, durfall: {:.2f}, durbed: {:.2f}".format(dict3['pid'][i], dict3['date'][i], eff, durafwake, dursleep, durawake, durfall, durbed))
                    except:
                        skip = True
                        break

                    break
                else:
                    i += 1
            
            if skip == True:
                continue

            # NEW
            I = "You are a personalized healthcare agent trained to predict PHQ-4 {} which ranges from 0 to 4 based on physiological data and user information.".format(SUBTASK)

            if SUBTASK == "depression":
                # I = "Given the user's context and sensor data, predict the PHQ-4 depression level (from 0 to 4)"
                A = "The predicted PHQ-4 {} score is {}.".format(SUBTASK, int(dep))
            elif SUBTASK == "anxiety":
                # I = "Given the      's context and sensor data, predict the PHQ-4 anxiety level (from 0 to 4)"
                A = "The predicted PHQ-4 {} score is {}.".format(SUBTASK, int(anx))
            
            Q = "The recent 14-days sensor readings show: [Steps] is {}. [Sleep] efficiency, duration the user stayed in bed after waking up, duration the user spent to sleep, duration the user stayed awake but still in bed, duration the user spent to fall asleep are {}, {}, {}, {}, {} mins in average; What would be the PHQ-4 {} score?".format(avgsumsteps, eff, durafwake, dursleep, durawake, durfall, SUBTASK)

            final_data.append({'task': SUBTASK,'instruction':I, 'input':Q, 'output':A})  
    final_data_globem.append(final_data)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:22<00:00,  5.58s/it]


In [6]:
final_data_globem

[[{'task': 'anxiety',
   'instruction': 'You are a personalized healthcare agent trained to predict PHQ-4 anxiety which ranges from 0 to 4 based on physiological data and user information.',
   'input': 'The recent 14-days sensor readings show: [Steps] is 16122.5. [Sleep] efficiency, duration the user stayed in bed after waking up, duration the user spent to sleep, duration the user stayed awake but still in bed, duration the user spent to fall asleep are 93.0, 0.0, 371.0, 28.0, 0.0 mins in average; What would be the PHQ-4 anxiety score?',
   'output': 'The predicted PHQ-4 anxiety score is 1.'},
  {'task': 'anxiety',
   'instruction': 'You are a personalized healthcare agent trained to predict PHQ-4 anxiety which ranges from 0 to 4 based on physiological data and user information.',
   'input': 'The recent 14-days sensor readings show: [Steps] is 15815.6666666667. [Sleep] efficiency, duration the user stayed in bed after waking up, duration the user spent to sleep, duration the user st

In [8]:
# ======================================================
# 1. 通用转换函数：把 GLOBEM (或其他) 数据转成 RLCR 格式
# ======================================================

def convert_to_rlcr_format(data_list, task_name, source_prefix):
    """
    将你的自定义数据格式转换为 RLCR 格式
    
    输入格式 (data_list 中每条 item):
    {
        'task': 'anxiety',
        'instruction': 'You are a personalized healthcare agent ...',
        'input': 'The recent 14-days sensor readings show: ...',
        'output': 'The predicted PHQ-4 anxiety score is 1.'
    }

    输出格式 (每条 item):
    {
        'problem': 'You are a ...\n\nThe recent 14-days sensor readings show: ...',
        'answer': '1',
        'source': 'GLOBEM-anxiety',
    }

    参数:
    - data_list: 某个子任务 (anxiety/depression) 的样本列表
    - task_name: 子任务名称，例如 "anxiety" 或 "depression"
    - source_prefix: 数据集前缀，比如 "GLOBEM" 或 "PMData"
    """
    converted_data = []
    
    for idx, item in enumerate(data_list):
        # 1. 将 instruction 和 input 合并为 problem
        problem = f"{item['instruction']}\n\n{item['input']}"
        
        # 2. 从 output 中提取数值答案
        output_text = item['output']

        # 优先匹配 "score is X" 或 "level is X" 的形式
        answer_match = re.search(r'(?:score|level) is (\d+)', output_text)
        if answer_match:
            answer = answer_match.group(1)
        else:
            # 如果上面的模式没匹配到，就退化为匹配任意数字
            answer_match = re.search(r'(\d+)', output_text)
            answer = answer_match.group(1) if answer_match else "0"
        
        # 3. 组装为 RLCR 需要的格式
        converted_item = {
            'problem': problem,
            'answer': answer,
            'source': f"{source_prefix}-{task_name}",
        }
        
        converted_data.append(converted_item)
    
    return converted_data


# ======================================================
# 2. 假设这里已经有 GLOBEM 的原始任务数据
#    final_data_globem: [anxiety_list, depression_list]
#    subtasks: ["anxiety", "depression"]
# ======================================================

# 如果你前面已经定义了，就不用再次定义：
# subtasks = ["anxiety", "depression"]
# final_data_globem = [...]  # 由你前面的 for SUBTASK in subtasks 那段代码生成

# 这里只是保证变量存在（如果你已经有，就注释掉下面两行）
# subtasks = ["anxiety", "depression"]
# final_data_globem = [...]  # 请用你实际的数据替换


# ======================================================
# 3. 进行 GLOBEM → RLCR 转换
# ======================================================

converted_datasets_globem = {}

for task_name, task_data in zip(subtasks, final_data_globem):
    print("\n" + "=" * 60)
    print(f"GLOBEM task_name: {task_name}")
    print(f"Original data size: {len(task_data)}")
    
    # 调用通用转换函数，source_prefix 使用 "GLOBEM"
    converted_data = convert_to_rlcr_format(
        data_list=task_data,
        task_name=task_name,
        source_prefix="GLOBEM"
    )
    
    print(f"Converted data size: {len(converted_data)}")
    if len(converted_data) > 0:
        print("\nFirst example:")
        print(f"Problem (first 200 chars): {converted_data[0]['problem'][:200]}...")
        print(f"Answer: {converted_data[0]['answer']}")
        print(f"Source: {converted_data[0]['source']}")
    
    converted_datasets_globem[task_name] = converted_data

print("\n" + "=" * 60)
print("✅ All GLOBEM tasks converted to RLCR format!")


# ======================================================
# 4. 保存每个 GLOBEM 子任务为单独 CSV 文件
# ======================================================

# Windows 路径建议写成原始字符串 r"..."
base_dir = r"C:\Users\35950\Downloads\Dataset-SLM\Processed_Data"

for task_name, data in converted_datasets_globem.items():
    ds = Dataset.from_list(data)
    csv_filename = fr"{base_dir}\GLOBEM_{task_name}.csv"
    ds.to_csv(csv_filename)
    print(f"💾 Saved {csv_filename} with {len(data)} rows.")


GLOBEM task_name: anxiety
Original data size: 1827
Converted data size: 1827

First example:
Problem (first 200 chars): You are a personalized healthcare agent trained to predict PHQ-4 anxiety which ranges from 0 to 4 based on physiological data and user information.

The recent 14-days sensor readings show: [Steps] is...
Answer: 1
Source: GLOBEM-anxiety

GLOBEM task_name: depression
Original data size: 1827
Converted data size: 1827

First example:
Problem (first 200 chars): You are a personalized healthcare agent trained to predict PHQ-4 depression which ranges from 0 to 4 based on physiological data and user information.

The recent 14-days sensor readings show: [Steps]...
Answer: 0
Source: GLOBEM-depression

✅ All GLOBEM tasks converted to RLCR format!


Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\GLOBEM_anxiety.csv with 1827 rows.


Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\GLOBEM_depression.csv with 1827 rows.


In [9]:
DATA_PATH = data_local_path_header + "Dataset-SLM/harvard_dataverse"
# SUBTASK = "activity"
# print("[INFO] Generating datasets for AW_FB (Harvard Dataverse) ...")
# print("[INFO] Subtask:", SUBTASK)
final_data_harvard_dataverse = []
with open('{}/aw_fb_data.csv'.format(DATA_PATH), newline='') as csvfile:
    # csv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    df = pd.read_csv('{}/aw_fb_data.csv'.format(DATA_PATH), quotechar='"')
    # final_data = []
    activities = []
    subtask1 = []
    subtask2 = []
    # for idx,row in enumerate(csv_reader):
    for idx, row in df.iterrows(): 
        if idx == 0:
            continue

        # try:
        # tmp = row[0].split(",")
        # print(tmp)
        age = int(row.iloc[2])
        gender = int(row.iloc[3])
        if gender == 1:
            gender = "M"
        elif gender == 0:
            gender = "F"
        height = str(row.iloc[4]) + " cm"
        weight = str(row.iloc[5]) + " kg"
        steps = float(row.iloc[6])
        heart_rate = float(row.iloc[7])
        calories = float(row.iloc[8])
        distance = float(row.iloc[9])
        entropy_heart = float(row.iloc[10])
        entropy_steps = float(row.iloc[11])
        resting_heart = float(row.iloc[12])
        corr_heart_steps = float(row.iloc[13])
        norm_heart = float(row.iloc[14])
        intensity_karvonen = float(row.iloc[15]) 
        sd_norm_heart = float(row.iloc[16])
        steps_times_distance = float(row.iloc[17]) 
        device = row.iloc[18]
        
        # if len(row) == 3:
        #     activity = row[0].split(",")[-1] + " " + row[1] + " " + row[2]
        # else:
        activity = row.iloc[19] # label

        activity = activity.replace('"', "")

        if activity == "Self Pace walk":
            met_value = 2
        elif activity == "Sitting":
            met_value = 1
        elif activity == "Lying":
            met_value = 0.5
        elif activity == "Running 7 METs":
            met_value = 7
        elif activity == "Running 5 METs":
            met_value = 5
        elif activity == "Running 3 METs":
            met_value = 3

        duration = calories / (met_value * float(row.iloc[5]) / 200)

        activities.append(activity)

        # I = "Answer this question truthfully"

        # if SUBTASK == "activity":
            # Q = "Predict the {} type among ['Self Pace Walk', 'Sitting', 'Lying', 'Running 7 METs', 'Sitting', 'Running 5 METs', 'Running 3 METs'] given the following information. [Age]: {}, [Gender]: {}, [Height]: {}, [Weight]: {}, [Steps]: {:.2f} steps, [Burned Calorories]: {:.2f} calories, [Heart Rate]: {:.2f} beats/min".format(SUBTASK, age, gender, height, weight, steps, calories, heart_rate)
            # A = "{}".format(activity.replace('\"', ''))
        
            # NEW
        I = "You are a personalized healthcare agent trained to predict the type of activity among ['Self Pace Walk', 'Sitting', 'Lying', 'Running 7 METs', 'Running 5 METs', 'Running 3 METs'] based on physiological data and user information."
        Q = "The recent sensor readings show: [Steps]: {:.2f} steps, [Burned Calorories]: {:.2f} calories, [Heart Rate]: {:.2f} beats/min; What would be the predicted activity type?".format(steps, calories, heart_rate)
        A = "The predicted activity type is {}".format(activity.replace('\"', ''))
        subtask1.append({'task':'activity','instruction':I, 'input':Q, 'output':A})
        # elif SUBTASK == "calories":
            # Q = "Predict the burned {} given the following information. [Age]: {}, [Gender]: {}, [Height]: {}, [Weight]: {}, [Steps]: {:.2f} steps, [Heart Rate]: {:.2f} beats/min".format(SUBTASK, age, gender, height, weight, steps, heart_rate)
            # A = "{}".format(calories)

            # NEW
        I = "You are a personalized healthcare agent trained to predict the calorie burn based on physiological data and user information."
        Q = "The recent sensor readings show: [Steps]: {:.2f} steps, [Heart Rate]: {:.2f} beats/min, [Duration]: {:.2f}, [Activity Type]: {}, [MET Value]: {}; What would be the predicted calorie burn?".format(steps, heart_rate, duration, activity, met_value)
        A = "The predicted calorie burn is {}.".format(calories)
        subtask2.append({'task': 'calories','instruction':I, 'input':Q, 'output':A})
    final_data_harvard_dataverse.append(subtask1)
    final_data_harvard_dataverse.append(subtask2)

In [10]:
final_data_harvard_dataverse

[[{'task': 'activity',
   'instruction': "You are a personalized healthcare agent trained to predict the type of activity among ['Self Pace Walk', 'Sitting', 'Lying', 'Running 7 METs', 'Running 5 METs', 'Running 3 METs'] based on physiological data and user information.",
   'input': 'The recent sensor readings show: [Steps]: 11.48 steps, [Burned Calorories]: 3.29 calories, [Heart Rate]: 78.45 beats/min; What would be the predicted activity type?',
   'output': 'The predicted activity type is Lying'},
  {'task': 'activity',
   'instruction': "You are a personalized healthcare agent trained to predict the type of activity among ['Self Pace Walk', 'Sitting', 'Lying', 'Running 7 METs', 'Running 5 METs', 'Running 3 METs'] based on physiological data and user information.",
   'input': 'The recent sensor readings show: [Steps]: 12.18 steps, [Burned Calorories]: 9.48 calories, [Heart Rate]: 78.54 beats/min; What would be the predicted activity type?',
   'output': 'The predicted activity typ

In [ ]:
def convert_to_rlcr_format_aw_fb(data_list, task_name, source_prefix="AW_FB"):
    """
    Convert AW_FB (Harvard Dataverse) samples into RLCR format.

    Input item example (activity):
    {
        'task': 'activity',
        'instruction': 'You are a personalized healthcare agent trained to predict ...',
        'input': 'The recent sensor readings show: [Steps]: 11.48 steps, ...',
        'output': 'The predicted activity type is Lying'
    }

    Input item example (calories):
    {
        'task': 'calories',
        'instruction': 'You are a personalized healthcare agent trained to predict ...',
        'input': 'The recent sensor readings show: [Steps]: 1000.0 steps, ...',
        'output': 'The predicted calorie burn is 123.45.'
    }

    RLCR output format (no index):
    {
        'problem': '<instruction>\\n\\n<input>',
        'answer': '<label or numeric string>',
        'source': 'AW_FB-activity' or 'AW_FB-calories'
    }
    """
    converted_data = []

    for item in data_list:
        problem = f"{item['instruction']}\n\n{item['input']}"
        output_text = item['output']

        if task_name == "activity":
            # Extract label after "is "
            # Example: "The predicted activity type is Lying"
            match = re.search(r"is\s+(.+)$", output_text)
            if match:
                answer = match.group(1).strip().rstrip(".")
            else:
                # Fallback to the whole output text if pattern does not match
                answer = output_text.strip()
        elif task_name == "calories":
            # Extract numeric value from "The predicted calorie burn is 123.45."
            match = re.search(r"([-+]?\d*\.?\d+)", output_text)
            if match:
                answer = match.group(1)
            else:
                # Fallback: set to "0"
                answer = "0"
        else:
            # Generic fallback: try to get any digit, or use full string
            match = re.search(r"([-+]?\d*\.?\d+)", output_text)
            answer = match.group(1) if match else output_text.strip()

        converted_item = {
            "problem": problem,
            "answer": answer,
            "source": f"{source_prefix}-{task_name}",
        }
        converted_data.append(converted_item)

    return converted_data


# ======================================================
# 3. Run conversion for both AW_FB subtasks and print a sample
# ======================================================

tasks = ["activity", "calories"]  # order matches final_data_harvard_dataverse
converted_datasets_harvard = {}

for task_name, task_data in zip(tasks, final_data_harvard_dataverse):
    print("\n" + "=" * 60)
    print(f"AW_FB task_name: {task_name}")
    print(f"Original data size: {len(task_data)}")

    converted_data = convert_to_rlcr_format_aw_fb(
        data_list=task_data,
        task_name=task_name,
        source_prefix="AW_FB"
    )

    print(f"Converted data size: {len(converted_data)}")
    if len(converted_data) > 0:
        print("\nFirst example:")
        print(f"Problem (first 200 chars): {converted_data[0]['problem'][:200]}...")
        print(f"Answer: {converted_data[0]['answer']}")
        print(f"Source: {converted_data[0]['source']}")

    converted_datasets_harvard[task_name] = converted_data

print("\n" + "=" * 60)
print("✅ All AW_FB tasks converted to RLCR format!")


# ======================================================
# 4. Save each AW_FB task as a CSV
# ======================================================

# Adjust this base directory to your own path convention
base_dir = r"C:\Users\35950\Downloads\Dataset-SLM\Processed_Data"

for task_name, data in converted_datasets_harvard.items():
    ds = Dataset.from_list(data)
    csv_filename = fr"{base_dir}\GLOBEM_{task_name}.csv"
    ds.to_csv(csv_filename)
    print(f"💾 Saved {csv_filename} with {len(data)} rows.")


AW_FB task_name: activity
Original data size: 6263
Converted data size: 6263

First example:
Problem (first 200 chars): You are a personalized healthcare agent trained to predict the type of activity among ['Self Pace Walk', 'Sitting', 'Lying', 'Running 7 METs', 'Running 5 METs', 'Running 3 METs'] based on physiologica...
Answer: Lying
Source: AW_FB-activity

AW_FB task_name: calories
Original data size: 6263
Converted data size: 6263

First example:
Problem (first 200 chars): You are a personalized healthcare agent trained to predict the calorie burn based on physiological data and user information.

The recent sensor readings show: [Steps]: 11.48 steps, [Heart Rate]: 78.4...
Answer: 3.28762546296296
Source: AW_FB-calories

✅ All AW_FB tasks converted to RLCR format!


Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\\Dataset-SLM\Processed_Data\AW_FB_activity.csv with 6263 rows.


Creating CSV from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\\Dataset-SLM\Processed_Data\AW_FB_calories.csv with 6263 rows.


In [12]:
SUBTASK = "stress_resilience" #"sleep_disorder"
print("[INFO] Subtask:", SUBTASK)
DATA_PATH = data_local_path_header + "Dataset-SLM/life_snaps"
#DATA_PATH = "data/life_snaps"

if SUBTASK == "stress_resilience":
    final_data_list = []

    # 1) read panas data
    with open('{}/scored_surveys/panas.csv'.format(DATA_PATH), newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')

        stress_resilience_list = []
        for idx,row in enumerate(csv_reader):
            if idx==0:
                continue
            # print("row:", row)
            tmp = row[0].split(",")
            user_id = tmp[1]
            date = datetime.strptime(tmp[3], '%Y-%m-%d')
            positive_affect_score = int(tmp[4]) # 10 ~ 50
            negative_affect_score = int(tmp[5]) # 10 ~ 50

            stress_resilience = positive_affect_score / negative_affect_score # 0.2 ~ 5
            stress_resilience_list.append({'user_id':user_id, 'date': date, 'stress_resilience': stress_resilience, 'positive_affect_score': positive_affect_score, 'negative_affect_score': negative_affect_score})

    # 2) read fitbit data
    with open('{}/csv_rais_anonymized/daily_fitbit_sema_df_unprocessed.csv'.format(DATA_PATH), newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        fitbit_list = []
        for idx,row in enumerate(csv_reader):
            if idx == 0:
                continue
            
            tmp = row[0].split(",")
            try:
                user_id = tmp[1]
                date = datetime.strptime(tmp[2], '%Y-%m-%d')
                activity_type = tmp[17]
                lightly_active_minutes = float(tmp[19])
                moderately_active_minutes = float(tmp[20])
                very_active_minutes = float(tmp[21])
                sleep_duration = float(tmp[26])
                sleep_efficiency = float(tmp[31])
                sleep_deep_ratio = float(tmp[32])
                sleep_light_ratio = float(tmp[34])
                sleep_rem_ratio = float(tmp[35])
                stress_score = float(tmp[8])
            except Exception as e:
                # print(e)
                continue

            # print("stress_score:", stress_score)

            fitbit_list.append({'user_id': user_id, 'date': date, 'activity_type':activity_type, 'lightly_active_minutes': lightly_active_minutes, 'moderately_active_minutes': moderately_active_minutes, 'very_active_minutes': very_active_minutes, 'sleep_duration': sleep_duration, 'sleep_efficiency': sleep_efficiency, 'sleep_deep_ratio':sleep_deep_ratio, 'sleep_light_ratio': sleep_light_ratio, 'sleep_rem_ratio':sleep_rem_ratio, 'stress_score':stress_score})

    # iterate through the label (stress resilience)
    for item1 in stress_resilience_list:
        user1 = item1['user_id']
        stress_resilience_index = item1['stress_resilience'] # what we want to predict
        tmp_positive_affect_score = item1['positive_affect_score']
        tmp_negative_affect_score = item1['negative_affect_score']

        tmp_activity_type = []
        tmp_lightly_active_minutes = []
        tmp_moderately_active_minutes = []
        tmp_very_active_minutes = []
        tmp_sleep_duration = []
        tmp_sleep_efficiency = []
        tmp_sleep_deep_ratio = []
        tmp_sleep_light_ratio = []
        tmp_sleep_rem_ratio = []
        tmp_stress_score = []

        for item2 in fitbit_list:
            user2 = item2['user_id']

            if user1 == user2:
                date1 = item1['date']
                date2 = item2['date']
                if (date1 > date2) and (date1 - date2) < timedelta(days=7):
                    # TODO: 1-week summary
                    tmp_activity_type.append(item2['activity_type'])
                    tmp_lightly_active_minutes.append(item2['lightly_active_minutes'])
                    tmp_moderately_active_minutes.append(item2['moderately_active_minutes'])
                    tmp_very_active_minutes.append(item2['very_active_minutes'])
                    tmp_sleep_duration.append(item2['sleep_duration'])
                    tmp_sleep_efficiency.append(item2['sleep_efficiency'])
                    tmp_sleep_deep_ratio.append(item2['sleep_deep_ratio'])
                    tmp_sleep_light_ratio.append(item2['sleep_light_ratio'])
                    tmp_sleep_rem_ratio.append(item2['sleep_rem_ratio'])
                    tmp_stress_score.append(item2['stress_score'])
        
        
        if len(tmp_activity_type) == 0:
            tmp_activity_type = ["N/A"]
        if len(tmp_lightly_active_minutes) == 0:
            tmp_lightly_active_minutes = [-1]
        if len(tmp_moderately_active_minutes) == 0:
            tmp_moderately_active_minutes = [-1]
        if len(tmp_very_active_minutes) == 0:
            tmp_very_active_minutes = [-1]
        if len(tmp_sleep_duration) == 0:
            tmp_sleep_duration = [-1]
        if len(tmp_sleep_efficiency) == 0:
            tmp_sleep_efficiency = [-1]
        if len(tmp_sleep_deep_ratio) == 0:
            tmp_sleep_deep_ratio = [-1]
        if len(tmp_sleep_light_ratio) == 0:
            tmp_sleep_light_ratio = [-1]
        if len(tmp_sleep_rem_ratio) == 0:
            tmp_sleep_rem_ratio = [-1]
        if len(tmp_stress_score) == 0:
            tmp_stress_score = [-1]
        
        I = "Answer this question truthfully"
        Q = "Given the following 7-days averaged data, predict the Stress Resilience Index (SRI) between 0.2 and 5. [Stress Score]: {} out of 100, [Positive Affect Score]: {} out of 50, [Negative Affect Score]: {} out of 50, [Lightly Active Minutes]: {}, [Moderately Active Minutes]: {}, [Very Active Minutes]: {}, [Sleep Efficiency]: {}, [Sleep Deep Ratio]: {}, [Sleep Light Ratio]: {}, [Sleep REM Ratio]: {}.".format(tmp_stress_score, tmp_positive_affect_score, tmp_negative_affect_score, tmp_lightly_active_minutes, tmp_moderately_active_minutes, tmp_very_active_minutes, tmp_sleep_efficiency, tmp_sleep_deep_ratio,tmp_sleep_light_ratio, tmp_sleep_rem_ratio)
        A = "{:.2f}".format(stress_resilience_index)

        # NEW
        # I = "You are a personalized healthcare agent trained to predict {} which ranges from 0.2 to 5 based on physiological data and user information.".format(SUBTASK)
        # Q = "The recent 7-days sensor readings show: [Stress Score]: {} out of 100, [Positive Affect Score]: {} out of 50, [Negative Affect Score]: {} out of 50, [Lightly Active Minutes]: {} minutes, [Moderately Active Minutes]: {} minutes, [Very Active Minutes]: {} minutes, [Sleep Efficiency]: {}, [Sleep Deep Ratio]: {}, [Sleep Light Ratio]: {}, [Sleep REM Ratio]: {}; What would be the predicted stress resilience index?".format(str(tmp_stress_score), str(tmp_positive_affect_score), str(tmp_negative_affect_score), str(tmp_lightly_active_minutes), str(tmp_moderately_active_minutes), str(tmp_very_active_minutes), str(tmp_sleep_efficiency), str(tmp_sleep_deep_ratio), str(tmp_sleep_light_ratio), str(tmp_sleep_rem_ratio))
        # A = "The predicted stress resilience index is {:.2f}.".format(stress_resilience_index)

        final_data_list.append({'instruction':I, 'input':Q, 'output':A})       

[INFO] Subtask: stress_resilience


In [13]:
final_data_list

[{'instruction': 'Answer this question truthfully',
  'input': 'Given the following 7-days averaged data, predict the Stress Resilience Index (SRI) between 0.2 and 5. [Stress Score]: [69.0, 82.0, 66.0, 74.0, 76.0, 80.0] out of 100, [Positive Affect Score]: 37 out of 50, [Negative Affect Score]: 14 out of 50, [Lightly Active Minutes]: [185.0, 147.0, 147.0, 178.0, 191.0, 142.0], [Moderately Active Minutes]: [37.0, 33.0, 21.0, 40.0, 49.0, 0.0], [Very Active Minutes]: [27.0, 33.0, 37.0, 45.0, 35.0, 0.0], [Sleep Efficiency]: [92.0, 96.0, 90.0, 93.0, 96.0, 94.0], [Sleep Deep Ratio]: [0.9102564102564102, 1.1153846153846154, 0.6075949367088608, 1.0129870129870129, 1.1168831168831168, 0.8181818181818182], [Sleep Light Ratio]: [0.6588628762541806, 1.129692832764505, 1.0745762711864406, 0.8344594594594594, 1.0716723549488054, 1.040677966101695], [Sleep REM Ratio]: [0.6020408163265306, 1.1354166666666667, 0.5463917525773195, 0.8085106382978723, 1.096774193548387, 1.1063829787234043].',
  'output':

In [19]:
final_data_lifesnaps_sleep_disorder = []

# read fitbit data
with open('{}/csv_rais_anonymized/daily_fitbit_sema_df_unprocessed.csv'.format(DATA_PATH), newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    fitbit_list = []
    for idx,row in enumerate(csv_reader):
        if idx == 0:
            continue
        
        tmp = row[0].split(",")
        try:
            user_id = tmp[1]
            date = datetime.strptime(tmp[2], '%Y-%m-%d')
 
            sleep_duration = float(tmp[26]) / 3600 / 1000
            minutes_awake = float(tmp[29])
            sleep_efficiency = float(tmp[31])
            sleep_deep_ratio = float(tmp[32])
            sleep_wake_ratio = float(tmp[33])
            sleep_light_ratio = float(tmp[34])
            sleep_rem_ratio = float(tmp[35])

            rmssd = float(tmp[5])
            spo2 = float(tmp[6])
            full_sleep_breathing_rate = float(tmp[7])
            bpm = float(tmp[18])
            resting_hr = float(tmp[25])
        
        except Exception as e:
            # print(e)
            continue

        # DSM-IV ICSD
        print("sleep_efficiency:", sleep_efficiency)
        print("minutes_awake:", minutes_awake)
        
        if sleep_efficiency < 85 and minutes_awake > 30:
            sleep_disorder = 1
        else:
            sleep_disorder = 0
        
        print("sleep_disorder:", sleep_disorder)
        print()

        I = "Answer this question truthfully"
        Q = "Given the following data, predict whether there exists sleep disorder (1) or not (0). [Sleep Duration]: {:.2f} hours, [Minutes Awake]: {} minutes, [Sleep Efficiency]: {}, [Sleep Deep Ratio]: {:.2f}, [Sleep Wake Ratio]: {:.2f}, [Sleep Light Ratio]: {:.2f}, [Sleep REM Ratio]: {:.2f}, [RMSSD]: {:.2f}, [SPO2]: {} %, [Full Sleep Breathing Rate]: {}, [BPM]: {:.2f}, [Resting Hour]: {:.2f} hours".format(sleep_duration, minutes_awake, sleep_efficiency, sleep_deep_ratio, sleep_wake_ratio, sleep_light_ratio, sleep_rem_ratio, rmssd, spo2, full_sleep_breathing_rate, bpm, resting_hr)
        A = "{}".format(sleep_disorder)

        # NEW
        # I = "You are a personalized healthcare agent trained to predict the {} which is either 1 (exist) or 0 (does not exist) based on physiological data and user information.".format(SUBTASK)
        # Q = "The recent sensor readings show: [Sleep Duration]: {:.2f} hours, [Minutes Awake]: {} minutes, [Sleep Efficiency]: {}, [Sleep Deep Ratio]: {:.2f}, [Sleep Wake Ratio]: {:.2f}, [Sleep Light Ratio]: {:.2f}, [Sleep REM Ratio]: {:.2f}, [RMSSD]: {:.2f}, [SPO2]: {} %, [Full Sleep Breathing Rate]: {}, [BPM]: {:.2f}, [Resting Hour]: {:.2f} hours; What would be the predicted sleep disorder?".format(sleep_duration, minutes_awake, sleep_efficiency, sleep_deep_ratio, sleep_wake_ratio, sleep_light_ratio, sleep_rem_ratio, rmssd, spo2, full_sleep_breathing_rate, bpm, resting_hr)
        # A = "The predicted sleep disorder is {}.".format(sleep_disorder)

        final_data_lifesnaps_sleep_disorder.append({'instruction':I, 'input':Q, 'output':A})        


sleep_efficiency: 93.0
minutes_awake: 76.0
sleep_disorder: 0

sleep_efficiency: 96.0
minutes_awake: 67.0
sleep_disorder: 0

sleep_efficiency: 96.0
minutes_awake: 42.0
sleep_disorder: 0

sleep_efficiency: 96.0
minutes_awake: 50.0
sleep_disorder: 0

sleep_efficiency: 94.0
minutes_awake: 59.0
sleep_disorder: 0

sleep_efficiency: 95.0
minutes_awake: 64.0
sleep_disorder: 0

sleep_efficiency: 96.0
minutes_awake: 78.0
sleep_disorder: 0

sleep_efficiency: 96.0
minutes_awake: 51.0
sleep_disorder: 0

sleep_efficiency: 90.0
minutes_awake: 56.0
sleep_disorder: 0

sleep_efficiency: 88.0
minutes_awake: 75.0
sleep_disorder: 0

sleep_efficiency: 93.0
minutes_awake: 40.0
sleep_disorder: 0

sleep_efficiency: 98.0
minutes_awake: 35.0
sleep_disorder: 0

sleep_efficiency: 94.0
minutes_awake: 30.0
sleep_disorder: 0

sleep_efficiency: 98.0
minutes_awake: 28.0
sleep_disorder: 0

sleep_efficiency: 96.0
minutes_awake: 34.0
sleep_disorder: 0

sleep_efficiency: 95.0
minutes_awake: 48.0
sleep_disorder: 0

sleep_ef

In [20]:
final_data_lifesnaps_sleep_disorder

[{'instruction': 'Answer this question truthfully',
  'input': 'Given the following data, predict whether there exists sleep disorder (1) or not (0). [Sleep Duration]: 9.12 hours, [Minutes Awake]: 76.0 minutes, [Sleep Efficiency]: 93.0, [Sleep Deep Ratio]: 0.99, [Sleep Wake Ratio]: 1.55, [Sleep Light Ratio]: 1.44, [Sleep REM Ratio]: 0.82, [RMSSD]: 31.45, [SPO2]: 94.6 %, [Full Sleep Breathing Rate]: 13.8, [BPM]: 71.12, [Resting Hour]: 63.70 hours',
  'output': '0'},
 {'instruction': 'Answer this question truthfully',
  'input': 'Given the following data, predict whether there exists sleep disorder (1) or not (0). [Sleep Duration]: 5.73 hours, [Minutes Awake]: 67.0 minutes, [Sleep Efficiency]: 96.0, [Sleep Deep Ratio]: 0.83, [Sleep Wake Ratio]: 1.37, [Sleep Light Ratio]: 0.79, [Sleep REM Ratio]: 0.38, [RMSSD]: 22.05, [SPO2]: 96.8 %, [Full Sleep Breathing Rate]: 12.8, [BPM]: 76.23, [Resting Hour]: 64.41 hours',
  'output': '0'},
 {'instruction': 'Answer this question truthfully',
  'input

In [ ]:
def convert_to_rlcr_format_lifesnaps(
    data_list,
    task_name,
    source_prefix="LifeSnaps",
):
    """
    Convert LifeSnaps samples into RLCR format.

    Input item example (stress_resilience):
    {
        'instruction': 'Answer this question truthfully',
        'input': 'Given the following 7-days averaged data, predict ...',
        'output': '2.64'
    }

    Input item example (sleep_disorder):
    {
        'instruction': 'Answer this question truthfully',
        'input': 'Given the following data, predict whether there exists sleep disorder (1) or not (0). ...',
        'output': '0'
    }

    RLCR output format (no index field):
    {
        'problem': '<instruction>\\n\\n<input>',
        'answer': '2.64' or '0',
        'source': 'LifeSnaps-stress_resilience' or 'LifeSnaps-sleep_disorder'
    }
    """
    converted_data = []

    for item in data_list:
        problem = f"{item['instruction']}\n\n{item['input']}"
        output_text = item["output"]

        # For stress_resilience: numeric value like "2.64"
        # For sleep_disorder: "0" or "1"
        # Extract the first numeric token from output_text
        match = re.search(r"[-+]?\d*\.?\d+", output_text)
        if match:
            answer = match.group(0)
        else:
            # Fallback: use the entire output string
            answer = output_text.strip()

        converted_item = {
            "problem": problem,
            "answer": answer,
            "source": f"{source_prefix}-{task_name}",
        }
        converted_data.append(converted_item)

    return converted_data


# ======================================================
# 2. Convert both LifeSnaps tasks
#    - final_data_list: stress_resilience
#    - final_data_lifesnaps_sleep_disorder: sleep_disorder
# ======================================================

converted_lifesnaps = {
    "stress_resilience": convert_to_rlcr_format_lifesnaps(
        final_data_list,
        task_name="stress_resilience",
        source_prefix="LifeSnaps",
    ),
    "sleep_disorder": convert_to_rlcr_format_lifesnaps(
        final_data_lifesnaps_sleep_disorder,
        task_name="sleep_disorder",
        source_prefix="LifeSnaps",
    ),
}

print("\n" + "=" * 60)
for task_name, data in converted_lifesnaps.items():
    print(f"[INFO] Task: {task_name}")
    print(f"[INFO] Converted size: {len(data)}")
    if len(data) > 0:
        print("[INFO] First RLCR example:")
        print("Problem (first 200 chars):", data[0]["problem"][:200], "...")
        print("Answer:", data[0]["answer"])
        print("Source:", data[0]["source"])
        print("-" * 40)


# ======================================================
# 3. Save each LifeSnaps task as a separate CSV
# ======================================================

base_dir = r"C:\Users\35950\Downloads\Dataset-SLM\Processed_Data"
os.makedirs(base_dir, exist_ok=True)

for task_name, data in converted_lifesnaps.items():
    ds = Dataset.from_list(data)
    csv_filename = fr"{base_dir}\LifeSnaps_{task_name}.csv"
    ds.to_csv(csv_filename)
    print(f"💾 Saved {csv_filename} with {len(data)} rows.")


[INFO] Task: stress_resilience
[INFO] Converted size: 268
[INFO] First RLCR example:
Problem (first 200 chars): Answer this question truthfully

Given the following 7-days averaged data, predict the Stress Resilience Index (SRI) between 0.2 and 5. [Stress Score]: [69.0, 82.0, 66.0, 74.0, 76.0, 80.0] out of 100, ...
Answer: 2.64
Source: LifeSnaps-stress_resilience
----------------------------------------
[INFO] Task: sleep_disorder
[INFO] Converted size: 889
[INFO] First RLCR example:
Problem (first 200 chars): Answer this question truthfully

Given the following data, predict whether there exists sleep disorder (1) or not (0). [Sleep Duration]: 9.12 hours, [Minutes Awake]: 76.0 minutes, [Sleep Efficiency]:  ...
Answer: 0
Source: LifeSnaps-sleep_disorder
----------------------------------------


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\LifeSnaps_stress_resilience.csv with 268 rows.


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

💾 Saved C:\Users\35950\Downloads\Dataset-SLM\Processed_Data\LifeSnaps_sleep_disorder.csv with 889 rows.
